In [1]:
import re

import numpy as np
import torch
import torch.utils.data as data_utils

from lolip.models.torch_utils.archs import *
from lolip.variables import auto_var
from lolip.utils import estimate_local_lip, estimate_local_lip_v2
from params import mnistLip, cifarLip
from utils import params_to_dataframe

In [ ]:
_, _, grid_params, _ = mnistLip()()
params_to_dataframe(grid_params, columns=['trn_acc', 'adv_trn_acc', 'avg_trn_lip', 'avg_tst_lip', 'history'], result_file_dir='./results/experiment01/')

In [9]:
_, _, grid_params, _ = cifarLip()()
print(grid_params
     )
params_to_dataframe(grid_params, columns=['trn_acc', 'history'], result_file_dir='./results/experiment01/')

I1225 13:39:02.514616 140075070584640 auto_var.py:261] Running time: 0.000087
I1225 13:39:02.606050 140075070584640 auto_var.py:261] Running time: 0.000079
I1225 13:39:02.690216 140075070584640 auto_var.py:261] Running time: 0.000042


[{'dataset': ['cifar10'], 'model': ['ce-tor-WRN_40_10', 'tradesce-tor-WRN_40_10', 'trades6ce-tor-WRN_40_10', 'advce-tor-WRN_40_10', 'llrce-tor-WRN_40_10'], 'eps': [0.031], 'norm': ['inf'], 'attack': ['pgd'], 'random_seed': [0]}]


I1225 13:39:02.773273 140075070584640 auto_var.py:261] Running time: 0.000111
I1225 13:39:02.855915 140075070584640 auto_var.py:261] Running time: 0.000041


ValueError: not enough values to unpack (expected 2, got 0)

In [16]:
def get_arch(path):
    return path.split("-")[path.split("-").index('tor')+1]

def get_ds(path):
    return path.split("-")[1]

def predict_real(model, X):
    X = X.transpose(0, 3, 1, 2)
    dataset = data_utils.TensorDataset(torch.from_numpy(X).float())
    loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=False, num_workers=2)
    ret = []
    for [x] in loader:
        ret.append(model(x.to('cuda')).detach().cpu().numpy())
    return np.concatenate(ret, axis=0)

def calc_lip(model, X, Xp, top_norm, btm_norm):
    top = np.linalg.norm(predict_real(model, X)-predict_real(model, Xp), ord=top_norm, axis=1)
    down = np.linalg.norm(X.reshape(len(Xp), -1)-Xp.reshape(len(Xp), -1), ord=btm_norm, axis=1)
    return top / (down+1e-6)

def get_lip(model, X, top_norm=2, btm_norm=2, epsilon=1.0):
    lip = estimate_local_lip_v2(model, X, top_norm=top_norm, btm_norm=btm_norm, epsilon=epsilon)
    lip_ = calc_lip(model, X, lip, top_norm=top_norm, btm_norm=btm_norm)
    return lip_

class ModelWrapper():
    def __init__(self, model):
        self.model = model
    def predict_real(self, X):
        return predict_real(self.model, X)
    def predict(self, X):
        return predict_real(self.model, X).argmax(axis=1)

In [ ]:
model_path = "./models/pgd-mnist-0.031-curece-tor-CNN001-inf-0.pt"
model = globals()[get_arch(model_path)](n_classes=10)
model.load_state_dict(torch.load(model_path))
model.eval()
model.cuda()

#auto_var.set_variable_value("norm", 'inf')
#auto_var.set_variable_value("eps", 0.031)
#trnX, trny, tstX, tsty = auto_var.get_var_with_argument("dataset", get_ds(model_path))
#attack_model = auto_var.get_var_with_argument("attack", "pgd", model=ModelWrapper(model))

from lolip.attacks.torch.projected_gradient_descent import ProjectedGradientDescent
nb_iter=10
attack_model = ProjectedGradientDescent(
    model_fn=model,
    norm=np.inf,
    eps=0.1,
    eps_iter=0.1*2/nb_iter,
    nb_iter=nb_iter,
)

adv_trnX = attack_model.perturb(trnX)
adv_tstX = attack_model.perturb(tstX)
print((ModelWrapper(model).predict(trnX) == trny).mean())
print((ModelWrapper(model).predict(adv_trnX) == trny).mean())

0.9529833333333333

1.0

In [17]:
ModelWrapper(model).predict(trnX)

array([5, 0, 4, ..., 5, 6, 8])

In [22]:
model(torch.Tensor(trnX[0:10].transpose(0, 3, 1, 2)))

tensor([[  1.1423, -15.6917,  -2.8912,  10.5858, -23.0581,  16.7434, -13.0156,
          -4.2783,  -6.4978,  -4.8898],
        [ 37.1476, -22.2011,  -6.6458,  -2.7661, -23.4233,  11.0323, -16.3949,
           0.3449,  -4.4311,  -9.6248],
        [ -9.1166,  -7.7489,  -7.2385,  -2.2442,  18.8515,  -1.9383,  -7.7448,
          -0.3562,  -2.3477,   9.9469],
        [-11.4429,  15.3514,  -1.6532,  -1.5187,  -1.6904,  -6.5238,  -9.2465,
           1.9417,   2.3212,  -1.5107],
        [-11.5020,  -6.4233, -10.5065,  -9.7716,  11.6808,  -0.2899, -14.0837,
           7.9795,   0.2486,  22.2252],
        [ -5.8696, -16.1103,  20.7923,  -2.4183, -11.1075,  -6.0231,  -9.4093,
           0.1035,  -1.5336,  -0.1243],
        [-12.1381,  18.7317,   1.8925,   1.5545,  -6.0536,  -4.0237,  -6.7087,
          -4.4256,  -0.2390,  -3.4156],
        [  1.6682, -14.4947,   6.3997,  30.9872, -23.0401,   4.2524, -27.6506,
         -12.2829,   5.5641,   0.2893],
        [-10.8424,  15.2722,  -2.2534,  -2.2202,